In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
import os
os.listdir("./asap-aes")

['valid_sample_submission_1_column.csv',
 'Essay_Set_Descriptions.zip',
 'training_set_rel3.xls',
 'valid_set.tsv',
 '.DS_Store',
 'valid_set.xls',
 'training_set_rel3.tsv',
 'training_set_rel3.xlsx',
 'valid_sample_submission_1_column_no_header.csv',
 'valid_sample_submission_5_column.csv',
 'Training_Materials',
 'valid_set.xlsx',
 'test_set.tsv',
 'Essay_Set_Descriptions',
 'Training_Materials.zip',
 'valid_sample_submission_2_column.csv']

In [20]:
valid_set = pd.read_csv("valid_set.tsv", sep='\t', encoding="latin-1")

In [21]:
valid_set['essay'][0]

'Dear @ORGANIZATION1, @CAPS1 more and more people start to use computers, goes more and more into the dark aged. While computer @MONTH1 be helpful in some cases, they are also making people excercize less, most @CAPS1 many people are nature, and more people are becoming awkward. So are computer advancement recently worth society becoming less interactive? The answer is no. With more thing to do on the computer people arent getting enough exercize to keep up a healthy regimens. Kids would rather waste time on the computer than go out play. @NUM1 years ago, when most that many people had computers, you could see people outside playing and having fun with each other, But now, when you look outside, no one is on the local playground having fun and exercising, they are on the internet. "@PERCENT1 of kids, who spend most of their time on the computer, are over-weight and unhealthy, compared to the @PERCENT2 of kids who are outside playing and a healthy weight." says @PERSON1, @CAPS2. So real

In [ ]:
text = "To Sherlock Holmes she is always THE woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer--excellent for drawing the veil from men's motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a goodie's distracting factor which might throw a doubt upon all his mental results."

In [22]:
valid_set['essay'][1]

"Dear @LOCATION1 Time @CAPS1 me tell you what I think of computers there is nothing wrong with people being on computers. I say this because us as kids really do need computers for projects and school work. Also because what if your parents want to move or get a better job but have no car they can use they're computer. Computers actually help the human mind. If you get bored then get on the computer sit back and relax while I explain. In addition I'm not saying that you can use your computer for everything say for an example your teacher gives you an assignment and you can't find it in your book your going to use the computer. If you need pictures for a project but your not an very good artist then you can use your computer. so at the end computer's aren't so bad after all. For an example my mom doesn't have a car or know how to drive. So when she needs to look for a job and no one is there to take her she get's on the internet and find one. Also when it was time for her to move she go

In [23]:
valid_set['essay'][2]

"Dear Local newspaper, Have you been spending a good portion of your time on computers? Do you think it is effecting you in anyway? Well in this writing piece I am going to tell you how I believe the computers effect you aand everyone else. My first piece of information for the way computers effect people is if they are on the omputer playing games, emailing or listening to music, that is more time you miss spending with thier family or going outside. If they miss time to spend with thier family then you could slowly start drifting apart from them, especially if they don't live near you. One day when that person passed away and you think back you are going to wish you weren't on the compuer three times they were over, you are going to wish you were down there spending time with them. If you miss going outside that is a whole new thing you are missing the sunshine, the snow or whatever kind of temparature there is outside and either way being outside in the fresh air is healthy and so m

In [25]:
valid_set

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN
...,...,...,...,...,...
4213,21933,8,Have you ever noticed that if two little kids...,24933,NaN
4214,21934,8,Laughter @CAPS1 I ...,24934,NaN
4215,21935,8,Laughter in @CAPS1 A laugh is not just an act...,24935,NaN
4216,21937,8,LAUGHTER @CAPS1 i was younger my friend live...,24937,NaN


In [15]:
valid_set['essay'] = [entry.lower() for entry in valid_set['essay']]
valid_set["essay"] = [word_tokenize(entry) for entry in valid_set["essay"]]
valid_set

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"[dear, @, organization1, ,, @, caps1, more, an...",1788,NaN
1,1789,1,"[dear, @, location1, time, @, caps1, me, tell,...",1789,NaN
2,1790,1,"[dear, local, newspaper, ,, have, you, been, s...",1790,NaN
3,1791,1,"[dear, readers, ,, @, caps1, you, imagine, how...",1791,NaN
4,1792,1,"[dear, newspaper, ,, i, strongly, believe, tha...",1792,NaN
...,...,...,...,...,...
4213,21933,8,"[have, you, ever, noticed, that, if, two, litt...",24933,NaN
4214,21934,8,"[laughter, @, caps1, i, was, a, little, girl, ...",24934,NaN
4215,21935,8,"[laughter, in, @, caps1, a, laugh, is, not, ju...",24935,NaN
4216,21937,8,"[laughter, @, caps1, i, was, younger, my, frie...",24937,NaN


In [16]:
valid_set["essay"].dropna(inplace=True)
valid_set["essay"] = [entry.lower() for entry in valid_set["essay"]]
valid_set["essay"] = [word_tokenize(entry) for entry in valid_set["essay"]]

AttributeError: 'list' object has no attribute 'lower'

In [9]:
valid_set['essay']

0       [dear, @, organization1, ,, @, caps1, more, an...
1       [dear, @, location1, time, @, caps1, me, tell,...
2       [dear, local, newspaper, ,, have, you, been, s...
3       [dear, readers, ,, @, caps1, you, imagine, how...
4       [dear, newspaper, ,, i, strongly, believe, tha...
                              ...                        
4213    [have, you, ever, noticed, that, if, two, litt...
4214    [laughter, @, caps1, i, was, a, little, girl, ...
4215    [laughter, in, @, caps1, a, laugh, is, not, ju...
4216    [laughter, @, caps1, i, was, younger, my, frie...
4217    [you, know, how, the, saying, goes, live, ,, l...
Name: essay, Length: 4218, dtype: object

In [17]:
np.random.seed(500)
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index, entry in enumerate(valid_set["essay"]):
    final_words = []
    word_lemmatized = WordNetLemmatizer()

    for word, tag in pos_tag(entry):
        if word not in stopwords.words("english") and word.isalpha():
            word_final = word_lemmatized.lemmatize(word, tag_map[tag[0]])
            final_words.append(word_final)
    valid_set.loc[index, "essay_final"] = str(final_words)


In [18]:
valid_set

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid,essay_final
0,1788,1,"[dear, @, organization1, ,, @, caps1, more, an...",1788,NaN,"['dear', 'people', 'start', 'use', 'computer',..."
1,1789,1,"[dear, @, location1, time, @, caps1, me, tell,...",1789,NaN,"['dear', 'time', 'tell', 'think', 'computer', ..."
2,1790,1,"[dear, local, newspaper, ,, have, you, been, s...",1790,NaN,"['dear', 'local', 'newspaper', 'spend', 'good'..."
3,1791,1,"[dear, readers, ,, @, caps1, you, imagine, how...",1791,NaN,"['dear', 'reader', 'imagine', 'life', 'would',..."
4,1792,1,"[dear, newspaper, ,, i, strongly, believe, tha...",1792,NaN,"['dear', 'newspaper', 'strongly', 'believe', '..."
...,...,...,...,...,...,...
4213,21933,8,"[have, you, ever, noticed, that, if, two, litt...",24933,NaN,"['ever', 'notice', 'two', 'little', 'kid', 'ma..."
4214,21934,8,"[laughter, @, caps1, i, was, a, little, girl, ...",24934,NaN,"['laughter', 'little', 'girl', 'know', 'much',..."
4215,21935,8,"[laughter, in, @, caps1, a, laugh, is, not, ju...",24935,NaN,"['laughter', 'laugh', 'action', 'also', 'meani..."
4216,21937,8,"[laughter, @, caps1, i, was, younger, my, frie...",24937,NaN,"['laughter', 'young', 'friend', 'live', 'ranch..."


In [7]:
valid_set["essay_final"]

0       ['dear', 'people', 'start', 'use', 'computer',...
1       ['dear', 'time', 'tell', 'think', 'computer', ...
2       ['dear', 'local', 'newspaper', 'spend', 'good'...
3       ['dear', 'reader', 'imagine', 'life', 'would',...
4       ['dear', 'newspaper', 'strongly', 'believe', '...
                              ...                        
4213    ['ever', 'notice', 'two', 'little', 'kid', 'ma...
4214    ['laughter', 'little', 'girl', 'know', 'much',...
4215    ['laughter', 'laugh', 'action', 'also', 'meani...
4216    ['laughter', 'young', 'friend', 'live', 'ranch...
4217    ['know', 'saying', 'go', 'live', 'laugh', 'lov...
Name: essay_final, Length: 4218, dtype: object

In [8]:
valid_set

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid,essay_final
0,1788,1,"[dear, @, organization1, ,, @, caps1, more, an...",1788,NaN,"['dear', 'people', 'start', 'use', 'computer',..."
1,1789,1,"[dear, @, location1, time, @, caps1, me, tell,...",1789,NaN,"['dear', 'time', 'tell', 'think', 'computer', ..."
2,1790,1,"[dear, local, newspaper, ,, have, you, been, s...",1790,NaN,"['dear', 'local', 'newspaper', 'spend', 'good'..."
3,1791,1,"[dear, readers, ,, @, caps1, you, imagine, how...",1791,NaN,"['dear', 'reader', 'imagine', 'life', 'would',..."
4,1792,1,"[dear, newspaper, ,, i, strongly, believe, tha...",1792,NaN,"['dear', 'newspaper', 'strongly', 'believe', '..."
...,...,...,...,...,...,...
4213,21933,8,"[have, you, ever, noticed, that, if, two, litt...",24933,NaN,"['ever', 'notice', 'two', 'little', 'kid', 'ma..."
4214,21934,8,"[laughter, @, caps1, i, was, a, little, girl, ...",24934,NaN,"['laughter', 'little', 'girl', 'know', 'much',..."
4215,21935,8,"[laughter, in, @, caps1, a, laugh, is, not, ju...",24935,NaN,"['laughter', 'laugh', 'action', 'also', 'meani..."
4216,21937,8,"[laughter, @, caps1, i, was, younger, my, frie...",24937,NaN,"['laughter', 'young', 'friend', 'live', 'ranch..."


In [14]:
# Train X is training set 
# Test X is testing set
### here
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(valid_set["essay_final"],valid_set["essay_set"],test_size=0.3)



In [15]:
Train_X

3807    ['patient', 'important', 'character', 'trait',...
3983    ['one', 'lay', 'good', 'idea', 'make', 'box', ...
2186    ['think', 'author', 'conclude', 'story', 'para...
3471    ['builder', 'empire', 'state', 'building', 'fa...
1300    ['author', 'inform', 'u', 'cycle', 'desert', '...
                              ...                        
3005    ['design', 'empire', 'state', 'build', 'builde...
3790    ['year', 'dad', 'mother', 'wait', 'ridicously'...
273     ['dear', 'supporter', 'technology', 'belive', ...
3905    ['patience', 'thing', 'little', 'luckly', 'peo...
951     ['clean', 'shelf', 'people', 'find', 'offensiv...
Name: essay_final, Length: 2952, dtype: object

In [16]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [17]:
Train_Y

array([6, 6, 3, ..., 0, 6, 1])

In [18]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(valid_set["essay_final"])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [19]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  97.47235387045814


In [20]:
Train_X_Tfidf

<2952x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 182937 stored elements in Compressed Sparse Row format>

In [21]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  99.28909952606635


In [9]:
valid = pd.read_csv("./asap-aes/valid_set.tsv", sep='\t', encoding="latin-1")
valid

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN
...,...,...,...,...,...
4213,21933,8,Have you ever noticed that if two little kids...,24933,NaN
4214,21934,8,Laughter @CAPS1 I ...,24934,NaN
4215,21935,8,Laughter in @CAPS1 A laugh is not just an act...,24935,NaN
4216,21937,8,LAUGHTER @CAPS1 i was younger my friend live...,24937,NaN


In [22]:
valid['essay'][1]

NameError: name 'valid' is not defined

In [65]:
valid_sett = pd.read_csv("valid_set.tsv", sep='\t', encoding="latin-1")

In [66]:
valid_sett

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN
...,...,...,...,...,...
4213,21933,8,Have you ever noticed that if two little kids...,24933,NaN
4214,21934,8,Laughter @CAPS1 I ...,24934,NaN
4215,21935,8,Laughter in @CAPS1 A laugh is not just an act...,24935,NaN
4216,21937,8,LAUGHTER @CAPS1 i was younger my friend live...,24937,NaN


In [76]:
valid_sett[valid_sett['domain2_predictionid'].notnull()]

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
589,4778,2,"Do u believe there are books, music, magizines...",6578,6579.0
590,4779,2,I strongly believe that there are some materia...,6580,6581.0
591,4780,2,"Do you think that certain books, movies, magaz...",6582,6583.0
592,4781,2,Censorship in libraries should definetly be al...,6584,6585.0
593,4782,2,Many books are helpful as you @MONTH1 know by ...,6586,6587.0
...,...,...,...,...,...
1184,5373,2,It is my opinion that content in libraries sho...,7768,7769.0
1185,5374,2,Libraries have been open to the public for yea...,7770,7771.0
1186,5375,2,A library is a place of information and wisdom...,7772,7773.0
1187,5376,2,"I believe that certain materials, such as book...",7774,7775.0


# Test Set

In [43]:
test_set = pd.read_csv("./asap-aes/test_set.tsv", sep='\t', encoding="latin-1")

In [44]:
test_set

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,2383,1,I believe that computers have a positive effec...,2383,NaN
1,2384,1,"Dear @CAPS1, I know some problems have came up...",2384,NaN
2,2385,1,"Dear to whom it @MONTH1 concern, Computers are...",2385,NaN
3,2386,1,"Dear @CAPS1 @CAPS2, @CAPS3 has come to my atte...",2386,NaN
4,2387,1,"Dear Local newspaper, I think that people have...",2387,NaN
...,...,...,...,...,...
4249,22238,8,Just One @CAPS1 @CAPS2 can bring people togeth...,25238,NaN
4250,22239,8,@CAPS8 Was my first time going to the band ca...,25239,NaN
4251,22240,8,Laughter is one of the most important things i...,25240,NaN
4252,22241,8,A @CAPS1 Of @C...,25241,NaN


In [59]:
test_set.max()

essay_id                                                            22242
essay_set                                                               8
essay                   Winter Hibiscus the author is trying to show...
domain1_predictionid                                                25242
domain2_predictionid                                                 8977
dtype: object

In [78]:
# test_set[test_set['essay_id'] > 5000]
test_set[test_set['domain2_predictionid'].notnull()]

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
594,5378,2,I think people should take books off the shelf...,7778,7779.0
595,5379,2,Whil I belive that they should remove some boo...,7780,7781.0
596,5380,2,"Removing all books, music, movies, and magazin...",7782,7783.0
597,5381,2,Censorship in libraries should be limited to a...,7784,7785.0
598,5382,2,Dear Katherine Paterson. There are alot of...,7786,7787.0
...,...,...,...,...,...
1189,5973,2,"No, I dont think books, and movies should come...",8968,8969.0
1190,5974,2,"No, i do not feel that libraries should take t...",8970,8971.0
1191,5975,2,I think that books that are tear up should be ...,8972,8973.0
1192,5976,2,Most books that are chosen at a local library ...,8974,8975.0


# Train Set

In [13]:
train_set = pd.read_csv("./asap-aes/training_set_rel3.tsv", sep='\t', encoding="latin-1")
train_bset

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,17,18,NaN,35,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12972,21628,8,I never understood the meaning laughter is th...,15,17,NaN,32,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20,26,40.0,40,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
12974,21630,8,Trippin' on fen...,20,20,NaN,40,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
train_set[train_set['essay_set'] == 1]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,1783,1,"Dear @CAPS1, @CAPS2 several reasons on way I t...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1779,1784,1,Do a adults and kids spend to much time on the...,3,4,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1780,1785,1,My opinion is that people should have computer...,4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1781,1786,1,"Dear readers, I think that its good and bad to...",1,1,NaN,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
train_set[train_set['domain1_score']==39]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
12296,20770,8,Laughter is indeed an important part of anyon...,21,18,NaN,39,NaN,NaN,NaN,...,3.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
12591,21137,8,My best friend @PERSON2 turned thirteen on a ...,19,20,NaN,39,NaN,NaN,NaN,...,5.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
12619,21170,8,I met my best friend @PERSON2 when we were in...,22,17,NaN,39,NaN,NaN,NaN,...,4.0,4.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12632,21186,8,In life @CAPS13 share many of your moments la...,21,18,NaN,39,NaN,NaN,NaN,...,5.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12773,21367,8,@CAPS1 @CAPS2 @CAPS3 in to the school's offi...,19,20,NaN,39,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
12842,21456,8,"""Laughter"" @CAPS1 you ever felt it...",21,18,NaN,39,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12855,21473,8,"When we all get together on my dad side, we ...",17,22,NaN,39,NaN,NaN,NaN,...,5.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
12943,21586,8,When I think of laughter you think of memorie...,18,21,NaN,39,NaN,NaN,NaN,...,5.0,5.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
train_set.min()

essay_id                                                          1
essay_set                                                         1
essay                                                           ...
rater1_domain1                                                    0
rater2_domain1                                                    0
rater3_domain1                                                   20
domain1_score                                                     0
rater1_domain2                                                    1
rater2_domain2                                                    1
domain2_score                                                     1
rater1_trait1                                                     0
rater1_trait2                                                     0
rater1_trait3                                                     0
rater1_trait4                                                     0
rater1_trait5                                   

In [40]:
train_set[(train_set['rater1_domain1'] == 0) & (train_set['essay_set'] == 1)]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6


In [79]:
train_set.dropna(subset=['rater1_domain2')]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
1783,2978,2,Certain materials being removed from libraries...,4,4,NaN,4,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1784,2979,2,Write a persuasive essay to a newspaper reflec...,1,2,NaN,1,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1785,2980,2,Do you think that libraries should remove cert...,2,3,NaN,2,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1786,2981,2,"In @DATE1's world, there are many things found...",4,4,NaN,4,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1787,2982,2,In life you have the 'offensive things'. The l...,4,4,NaN,4,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3578,4773,2,The author is writting about taking books off ...,3,2,NaN,3,3.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3579,4774,2,"I do not think that materials, such as books, ...",3,3,NaN,3,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3580,4775,2,"Yes we should keep the books,music,movies,an m...",2,2,NaN,2,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3581,4776,2,"I do believe that book, magazines, music, mov...",3,4,NaN,3,4.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
x = training_set_rel3.dropna(subset=['rater3_domain1'])
x.dropna(subset=['rater1_domain2'])

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6


In [28]:
training_set_rel3[training_set_rel3['rater3_domain1'].isnull()]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12969,21623,8,laughter is an important part of any kind of ...,18,17,NaN,35,NaN,NaN,NaN,...,4.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12971,21626,8,In most stories mothers and daughters are eit...,17,18,NaN,35,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12972,21628,8,I never understood the meaning laughter is th...,15,17,NaN,32,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12974,21630,8,Trippin' on fen...,20,20,NaN,40,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# training_set_rel3.dropna(subset=['rater1_trait1'])
training_set_rel3.filter(regex="rater1_trait*")
# training_set_rel3.filter(regex="rater*_trait*")

,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
12971,4.0,3.0,4.0,4.0,4.0,3.0
12972,3.0,3.0,4.0,3.0,3.0,3.0
12973,4.0,4.0,4.0,4.0,4.0,4.0
12974,4.0,4.0,4.0,4.0,4.0,4.0


# Training Set Testing

In [406]:
train_set = pd.read_csv("./asap-aes/training_set_rel3.tsv", sep='\t', encoding="latin-1")

In [407]:
train_set = train_set[train_set['essay_set'] == 2]
train_set = train_set.reset_index()

In [408]:
train_set.drop(train_set.columns[0], axis=1, inplace=True)
train_set.drop(train_set.columns[7:], axis=1, inplace=True)
train_set.drop(train_set.columns[5], axis=1, inplace=True)

In [409]:
train_set['essay'] = [entry.lower() for entry in train_set['essay']]
train_set["essay"] = [word_tokenize(entry) for entry in train_set["essay"]]

In [410]:
train_set['avg_score'] = (train_set['rater1_domain1'] + train_set['rater2_domain1']) / 2
train_set['avg_score'] = train_set['avg_score'].apply(np.ceil).astype(int)

In [411]:
np.random.seed(500)
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
count = 0
for index, entry in enumerate(train_set['essay']):
    final_words = []
    word_lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words("english") and word.isalpha():
            word_final = word_lemmatized.lemmatize(word, tag_map[tag[0]])
            final_words.append(word_final)

    train_set.loc[index, "essay_final"] = str(final_words)
train_set.drop(train_set.columns[2], axis=1, inplace=True)

In [412]:
train_set

,essay_id,essay_set,rater1_domain1,rater2_domain1,domain1_score,avg_score,essay_final
0,2978,2,4,4,4,4,"['certain', 'material', 'remove', 'library', '..."
1,2979,2,1,2,1,2,"['write', 'persuasive', 'essay', 'newspaper', ..."
2,2980,2,2,3,2,3,"['think', 'library', 'remove', 'certain', 'mat..."
3,2981,2,4,4,4,4,"['world', 'many', 'thing', 'find', 'offensive'..."
4,2982,2,4,4,4,4,"['life', 'thing', 'little', 'stuff', 'get', 's..."
...,...,...,...,...,...,...,...
1795,4773,2,3,2,3,3,"['author', 'writting', 'take', 'book', 'adult'..."
1796,4774,2,3,3,3,3,"['think', 'material', 'book', 'music', 'movie'..."
1797,4775,2,2,2,2,2,"['yes', 'keep', 'book', 'music', 'movie', 'mag..."
1798,4776,2,3,4,3,4,"['believe', 'book', 'magazine', 'music', 'movi..."


In [413]:
# splitting into training and testing set
train_essay, test_essay, train_label, test_label = model_selection.train_test_split(train_set['essay_final'], train_set['avg_score'], test_size=0.3)

In [414]:
# transform the avg score into label of 0,1,2,3....
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)

In [401]:
train_label

array([3, 3, 3, ..., 2, 2, 3])

In [402]:
test_label

array([3, 2, 3, 2, 3, 3, 1, 3, 3, 2, 1, 1, 1, 1, 2, 4, 3, 3, 4, 4, 2, 2,
       3, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3, 3, 0, 2, 2,
       3, 3, 3, 4, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2,
       2, 4, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 1, 3, 3, 3, 3, 3, 2, 3, 2,
       2, 3, 3, 2, 5, 3, 3, 3, 2, 2, 4, 2, 2, 3, 2, 4, 2, 3, 3, 3, 2, 2,
       2, 3, 2, 1, 3, 2, 3, 4, 2, 2, 3, 2, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3,
       1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 1, 2, 2, 2, 3, 3, 3, 3, 1, 1, 2, 3, 3, 2, 3, 2, 1,
       2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 1, 3, 2, 2, 3, 3, 2,
       2, 2, 3, 4, 3, 3, 2, 2, 2, 4, 1, 3, 4, 2, 3, 3, 1, 2, 3, 3, 2, 3,
       3, 2, 2, 3, 3, 3, 3, 3, 2, 1, 3, 3, 3, 2, 3, 2, 2, 2, 3, 2, 3, 2,
       3, 3, 3, 5, 3, 2, 3, 2, 4, 2, 2, 3, 2, 1, 4, 3, 4, 2, 2, 2, 3, 2,
       3, 4, 3, 2, 2, 2, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 0, 3, 2, 2, 2,
       3, 3, 2, 3, 3, 1, 2, 3, 3, 4, 2, 0, 4, 3, 2,

In [415]:
# transform essay into matrix
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(train_set["essay_final"])
train_essay_vect = Tfidf_vect.transform(train_essay)
test_essay_vect = Tfidf_vect.transform(test_essay)

In [384]:
test_essay

1131    ['there', 'be', 'many', 'different', 'perspeti...
834     ['if', 'i', 'think', 'that', 'something', 'be'...
1100    ['i', 'do', 'not', 'believe', 'that', 'censors...
70      ['i', 'believe', 'that', 'all', 'the', 'book',...
863     ['book', 'movie', 'and', 'music', 'should', 'n...
                              ...                        
1674    ['censorship', 'to', 'most', 'people', 'mean',...
409     ['any', 'one', 'have', 'the', 'right', 'to', '...
649     ['i', 'belive', 'that', 'the', 'view', 'on', '...
1186    ['i', 'go', 'to', 'the', 'libbrary', 'every', ...
69      ['many', 'people', 'be', 'very', 'stern', 'abo...
Name: essay_final, Length: 540, dtype: object

In [391]:
test_label

array([3, 3, 3, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 5, 3, 3, 3, 1, 2, 3, 3, 3,
       3, 2, 2, 3, 3, 2, 2, 1, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3,
       2, 3, 4, 2, 3, 3, 3, 3, 3, 4, 1, 3, 2, 3, 2, 4, 2, 1, 2, 4, 2, 3,
       3, 3, 3, 3, 0, 3, 2, 2, 2, 2, 2, 3, 3, 2, 2, 4, 3, 3, 3, 4, 2, 2,
       2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 2,
       3, 2, 3, 2, 3, 3, 2, 4, 2, 3, 3, 3, 1, 2, 3, 2, 2, 2, 2, 3, 2, 3,
       2, 3, 3, 3, 2, 3, 2, 3, 4, 2, 2, 3, 3, 3, 2, 2, 2, 3, 0, 3, 3, 4,
       3, 3, 2, 3, 3, 2, 2, 4, 4, 2, 2, 3, 3, 3, 3, 3, 2, 4, 1, 2, 3, 3,
       3, 3, 2, 3, 5, 3, 3, 3, 1, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2,
       3, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 1, 3, 3, 2, 3, 3, 2, 3, 3, 3,
       3, 2, 3, 1, 3, 2, 3, 4, 3, 1, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 2, 2,
       3, 1, 2, 1, 2, 3, 2, 3, 2, 2, 3, 2, 2, 3, 0, 3, 1, 2, 3, 3, 2, 3,
       3, 3, 3, 1, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 3, 2, 2, 4, 3, 4, 2, 3,
       2, 3, 1, 3, 2, 3, 3, 2, 2, 2, 3, 2, 3, 4, 3,

In [385]:
train_essay

1066    ['author', 'katherine', 'paterson', 'once', 'c...
59      ['when', 'you', 'be', 'a', 'child', 'how', 'mu...
303     ['at', 'a', 'local', 'library', 'you', 'can', ...
759     ['should', 'book', 'that', 'be', 'offensive', ...
411     ['people', 'should', 'not', 'take', 'all', 'th...
                              ...                        
1345    ['should', 'there', 'be', 'cenorship', 'in', '...
284     ['how', 'many', 'of', 'you', 'have', 'ever', '...
1202    ['i', 'believe', 'that', 'many', 'book', 'in',...
1250    ['a', 'a', 'student', 'at', 'a', 'public', 'sc...
114     ['do', 'you', 'think', 'there', 'should', 'be'...
Name: essay_final, Length: 1260, dtype: object

In [386]:
# fitting training set into naive bayes
naive = naive_bayes.MultinomialNB()
naive.fit(train_essay_vect, train_label)

# fitting testing set on NB classifier
predictions_NB = naive.predict(test_essay_vect)
print(accuracy_score(predictions_NB, test_label)*100)
# predictions_NB

48.7037037037037


In [388]:
predictions_NB

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [389]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='rbf', degree=2, gamma='auto')
SVM.fit(train_essay_vect, train_label)
predictions_SVM = SVM.predict(test_essay_vect)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, test_label)*100)

SVM Accuracy Score ->  48.7037037037037


In [369]:
predictions_SVM

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [390]:
c = 0
for value in test_label:
    if value == 3:
        c += 1
t = len(test_label)
print(c / t * 100)


48.7037037037037
